In [3]:
import pandas as pd
import boto3
import json

#### Get data stored from Amazon S3

In [4]:
aws_access_key_id = ''
aws_secret_access_key = ''
region_name = 'ap-southeast-1'

s3 = boto3.client('s3',
                  aws_access_key_id=aws_access_key_id,
                  aws_secret_access_key=aws_secret_access_key,
                  region_name=region_name)
bucket_name = 'uitiotprocess'
prefix = 'sensor-data/'

response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
file_keys = [obj['Key'] for obj in response.get('Contents', [])]

latest_file_key = file_keys[-2]
crop_data_key = file_keys[-1]

obj = s3.get_object(Bucket=bucket_name, Key=latest_file_key)
obj_crop_data = s3.get_object(Bucket=bucket_name, Key=crop_data_key)

data = json.loads(obj['Body'].read().decode('utf-8'))
crop_data = json.loads(obj_crop_data['Body'].read().decode('utf-8'))


# print(f"Data trong file {latest_file_key}:")
# print(json.dumps(data, indent=2))
device_id = data.get('device_id')
timestamp = data.get('timestamp')
temperature = data.get('temperature')
humidity = data.get('humidity')
rainfall = data.get('rainfall')
n = data.get('n')
p = data.get('p')
k = data.get('k')
ph = data.get('ph')
df = pd.DataFrame(crop_data)

# print(device_id, timestamp, temperature, humidity, rainfall, n, p, k, ph)

#### Data preprocessing

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier

df['Sum'] = df[['N', 'P', 'K']].sum(axis=1)

# Define recommended actions based on rules (example rules)
def get_recommended_action(row):
    if row['humidity'] < 70:
        return 'Provide More Humidity' # Bật quạt
    if row['rainfall'] < 150 and row['temperature'] < 25:
        return 'Increase Watering' # Tưới nước
    if row['N'] < 3:
        return 'Add Fertilizer'
    if row['P'] < 3:
        return 'Add Phosphorus'

    # if row['N'] > 4 and row['P'] > 4 and row['K'] > 4:
    #     return 'Optimal Nutrient Levels'
    return 'No Specific Action'


# Apply the rules to generate recommended actions
df['Recommended_Action'] = df.apply(get_recommended_action, axis=1)

In [6]:
df.dropna(inplace=True)
label_encoder = LabelEncoder()
df['Recommended_Action_Encoded'] = label_encoder.fit_transform(df['Recommended_Action'])

X = df[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']]
y = df['Recommended_Action_Encoded']

#### Training model (Random Forest + Random SearchCV)

In [7]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import warnings
warnings.filterwarnings("ignore")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_dist = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

clf = RandomForestClassifier()
randomized_search = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)

randomized_search.fit(X_train, y_train)
best_params = randomized_search.best_params_
# print("Best Hyperparameters:", best_params)

best_clf = RandomForestClassifier(**best_params)
best_clf.fit(X_train, y_train)


y_pred = best_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
# print("RFC Accuracy:", accuracy)

labels_in_test = np.unique(y_test)
target_names = label_encoder.inverse_transform(labels_in_test)

# print(classification_report(y_test, y_pred, labels=labels_in_test, target_names=target_names, zero_division=1))

In [11]:
df.head(2)

,N,P,K,temperature,humidity,ph,rainfall,label,Sum,Recommended_Action,Recommended_Action_Encoded
0,90,42,43,20.879744,82.002744,6.502985,202.935536,tomato,175,No Specific Action,2
1,85,58,41,21.770462,80.319644,7.038096,226.655537,tomato,184,No Specific Action,2


#### Predicting label

In [15]:
# temperature = 15
# humidity = 90
# rainfall = 130
# n = 80
# p = 40
# k = 30
# ph = 5.8

In [16]:
X_new = np.array([[n, p, k, temperature, humidity, ph, rainfall]])
y_predict = best_clf.predict(X_new)
encoded_label = y_predict[0]
original_label = label_encoder.inverse_transform([encoded_label])[0]
# print("Action:", original_label)
print(encoded_label)

1


#### Send predicted result to ThingSpeak

In [9]:
import requests

write_api_key = ''
field_number = 1
value = encoded_label

url = f'https://api.thingspeak.com/update?api_key={write_api_key}&field{field_number}={value}'
response = requests.get(url)

if response.status_code == 200:
    print("✅ Đã gửi lên ThingSpeak:", value)
else:
    print("❌ Lỗi khi gửi dữ liệu:", response.text)

✅ Đã gửi lên ThingSpeak: 2


In [10]:
import requests

WRITE_API_KEY_CH1 = ''

action_code = encoded_label
action_map = {
    0: "Add Fertilizer",
    1: "Increase Watering",
    2: "No Specific Action",
    3: "Provide More Humidity"
}
action_message = action_map.get(action_code, "Unknown Action")

url = f"https://api.thingspeak.com/update?api_key={WRITE_API_KEY_CH1}&field1={action_message}"
response = requests.get(url)
print(f"✅ Sent to ThingSpeak: {response.status_code} | Entry ID: {response.text}")

✅ Sent to ThingSpeak: 200 | Entry ID: 0
